In [2]:
# list and tuple
l = [1, 2, 3]
print(l.__sizeof__())
tup = (1, 2, 3)
print(tup.__sizeof__())

64
48


In [6]:
# 列表是动态的，需存储指针。对应int型，8字节。
l = []
print(l.__sizeof__())   # 空列表存储空间为40字节
l.append(1)
print(l.__sizeof__())   # 加入1后，列表分配可以存储4个元素的空间 （70-40）/8 = 4
l.append(2)
print(l.__sizeof__())
l.append(3)
print(l.__sizeof__())
l.append(4)
print(l.__sizeof__())
l.append(5)
print(l.__sizeof__())   # 加入5，列表空间不足，需额外分配可以存储4个元素的空间

40
72
72
72
72
104


In [7]:
# 两种创建空列表的区别
# 区别主要在于list()是一个function call，Python的function call会创建stack，并且进行# 一系列参数检查的操作，比较expensive，反观[]是一个内置的C函数，可以直接被调用，因此效# 率高。

# empty_list = list()
# empty_list_2 = []


In [8]:
# dict and set
# 字典的性能更优，特别是对于查找、添加和删除操作。字典都能在常数时间复杂度内完成
# 区别在于集合没有键值对，是一系列无序的、唯一的元素组合
d1 = {'name': 'jason', 'age': 20, 'gender': 'male'}
d2 = dict({'name': 'jason', 'age': 20, 'gender': 'male'})
d3 = dict([('name', 'jason'), ('age', 20), ('gender', 'male')])
d4 = dict(name='jason', age=20, gender='male')
d1 == d2 == d3 ==d4

True

In [9]:
s1 = {1, 2, 3}
s2 = set([1, 2, 3])
s1 == s2

True

In [11]:
# 集合并不支持索引操作，因为集合本质是一个哈希表，和列表一样
print(1 in s1)
print(4 in s1)
s1.add(4)
print(4 in s1)
s1.remove(4)
print(4 in s1)

True
False
True
False


In [12]:
# 不同于其他数据结构，字典和集合的内部结构都是一张哈希表。对于字典而言，这张表存储了哈希值、键和值这3个元素。而对于集合而言，区别就是哈希表内没有键和值的配对，只有单一的元素了。插入、查找和删除的时间复杂度为O（1）
# 字典在python3.7+ 是有序的数据结构，而集合是无序的。字典和集合通常运用在对元素的高效查找、去重等场景

In [14]:
# python字符串是不可变的，每次改变字符串，往往需要O（n）的时间复杂度。
# python2.5后，字符串拼接操作（str1 += str2）会先检测str1还有没有其他的引用。如果没有的话，会尝试原地扩充字符串buffer的大小，而不是重新分配一块内存来创建新的字符串并拷贝。所以时间复杂度就仅为O（n）
s = ''
for n in range(10000):
    s += str(n)

In [22]:
import time

def func_perf(second:int):
    start_time = time.perf_counter()
    s = ''
    for n in range(second):
        s += str(n)
    end_time = time.perf_counter()
    print(f'{second} : {end_time - start_time}')

    start_time = time.perf_counter()
    l = []
    for n in range(second):
        l.append(str(n))
    s = ''.join(l)
    end_time = time.perf_counter()
    print(f'{second} : {end_time - start_time}')

    start_time = time.perf_counter()
    s = " ".join(map(str, range(second)))
    end_time = time.perf_counter()
    print(f'{second} : {end_time - start_time}')

func_perf(100)
func_perf(10000)
func_perf(1000000)

100 : 0.0008474999995087273
100 : 0.0002561999999670661
100 : 6.590000066353241e-05
10000 : 0.007062600000608654
10000 : 0.004774300000462972
10000 : 0.002546000000620552
1000000 : 2.163005499999599
1000000 : 0.28831799999989016
1000000 : 0.22151579999990645


In [52]:
from collections import Counter
import re

def read_file(filename:str):
    try:
        words = []
        with open(filename, 'r') as f:
            while True:
                line = f.readline()
                if line:
                    yield line
                else:
                    break
    except:
        raise Exception(f'{filename} not found')

def parse(line:str):
    text = re.sub(r'[^\w]', ' ', line).lower()

    words_list = text.split(' ')
    words_list = filter(None, words_list)
    return words_list

def word_count(filename:str, most_common:int):
    words_list = []
    for line in read_file(filename):
        words = parse(line)
        words_list.extend(words)
    return Counter(words_list).most_common(most_common)

word_count('I_have_a_dream.txt', most_common=10)

[('the', 103),
 ('of', 99),
 ('to', 59),
 ('and', 53),
 ('a', 37),
 ('we', 33),
 ('be', 33),
 ('will', 27),
 ('that', 24),
 ('is', 23)]

In [49]:
# 使用enumerate遍历集合，不仅返回每个元素，并且返回其对应索引
word_count = [('the', 103),('of', 99),('to', 59),('and', 53),('a', 37),('we', 33),('be', 33),('will', 27),('that', 24),('is', 23)]
for index, item in enumerate(word_count):
    print(f'{index} {item[0]} {item[1]}')

0 the 103
1 of 99
2 to 59
3 and 53
4 a 37
5 we 33
6 be 33
7 will 27
8 that 24
9 is 23


In [53]:
# 对比range（）和while循环的效率
# range（）函数是直接由C语言写的，调用速度非常快
# while循环中的“i += 1”操作，是通过python的解释器间接调用底层的C语言实现的，并且这个# 简单的操作又涉及到对象的创建和删除。（因为 i 是整型，是 immutable，i += 1 相当于 i # = new int(i + 1)）

In [54]:
attributes = ['name', 'dob', 'gender']
values = [['jason', '2000-01-01', 'male'],
['mike', '1999-01-01', 'male'],
['nancy', '2001-02-01', 'female']
]


In [63]:
[dict(zip(attributes, value)) for value in values]

[{'name': 'jason', 'dob': '2000-01-01', 'gender': 'male'},
 {'name': 'mike', 'dob': '1999-01-01', 'gender': 'male'},
 {'name': 'nancy', 'dob': '2001-02-01', 'gender': 'female'}]

In [ ]:
# closure 闭包
def nth_power(exponent):
    def exponent_of(base):
        return base * exponent
    return exponent_of

square = nth_power(2)
cube = nth_power(3)

print(square(2))
print(cube(2))